# Full pipeline in action (beta version)
----------

This notebook contains a set up of the full pipeline

**Inputs:**

- 
**Outputs:**
- results.csv (triple per line)

##### Table of contents:
1. Setup
2. Scraping
3. NLP Preprocessing
4. Information extraction
    - Named entity recognition
    - Entity linking
    - Relation extraction Linking
    - Relation linking
----------

### 1. Setup

##### Imports, settings & constants

In [3]:
import spacy
import pandas as pd
import numpy as np
import re
import requests
import sys
from tqdm import tqdm
from pathlib import Path



sys.path.insert(0, "../")
# Imports for NLP
from nlp import beautifulsoup as bsp
from nlp import nlp_preprocessing as nlp_prep
from nlp.read_warc2 import read_warc
from corpus_processing import ner 
from corpus_processing import relation_extraction as re
from corpus_processing import relation_linking as rl
from corpus_processing import entity_linking as el 
from corpus_processing import coref as cr

OSError: [E050] Can't find model 'en_core_web_trf'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
DIR_DATA = Path("../data/warcs")
FNAME_WARC = "sample.warc.gz"

### 1. Reading WARC files

In [ ]:
warc_df = read_warc(DIR_DATA / FNAME_WARC)

reading warc file...
warc list ['clueweb12-0000tw-00-00006' 'clueweb12-0000tw-00-00043'
 'clueweb12-0000tw-00-00044' 'clueweb12-0000tw-00-00101'
 'clueweb12-0000tw-00-00129' 'clueweb12-0000tw-00-00130'
 'clueweb12-0000tw-00-00141' 'clueweb12-0000tw-00-00160'
 'clueweb12-0000tw-00-00161' 'clueweb12-0000tw-00-00162'
 'clueweb12-0000tw-00-00199' 'clueweb12-0000tw-00-00205'
 'clueweb12-0000tw-00-00210' 'clueweb12-0000tw-00-00212'
 'clueweb12-0000tw-00-00214' 'clueweb12-0000tw-00-00829'
 'clueweb12-0000tw-00-00833' 'clueweb12-0000tw-00-00834'
 'clueweb12-0000tw-00-00835' 'clueweb12-0000tw-00-00836'
 'clueweb12-0000tw-00-00837' 'clueweb12-0000tw-00-00843'
 'clueweb12-0000tw-00-00844' 'clueweb12-0000tw-00-00877'
 'clueweb12-0000tw-00-00879' 'clueweb12-0000tw-00-00880'
 'clueweb12-0000tw-00-00881' 'clueweb12-0000tw-00-00941'
 'clueweb12-0000tw-00-00942' 'clueweb12-0000tw-00-00943'
 'clueweb12-0000tw-00-00944' 'clueweb12-0000tw-00-00946']


In [ ]:
warc_df

,HTML_DOC,WARC-TREC-ID
0,"<html>\r\n\r\n<head>\r\n<meta http-equiv=""Cont...",clueweb12-0000tw-00-00006
1,"<html>\n\n<head>\n\n<style type=""text/css"">\n\...",clueweb12-0000tw-00-00043
2,"<html>\n<head>\n<style type=""text/css"">\n#bott...",clueweb12-0000tw-00-00044
3,<html>\n<head>\n\n<title>DHP Concerts Bristol ...,clueweb12-0000tw-00-00101
4,<html>\n<head>\n <title>12 Tourist Spots in ...,clueweb12-0000tw-00-00129
5,<html>\r\n<head>\r\n<title>Saving Money Buying...,clueweb12-0000tw-00-00130
6,<html>\n<head>\n <title>Freelance Web Desig...,clueweb12-0000tw-00-00141
7,"<html>\n<head>\n<meta http-equiv=""Content-Type...",clueweb12-0000tw-00-00160
8,"<html>\n<head>\n <meta http-equiv=""Content-...",clueweb12-0000tw-00-00161
9,"<html>\n<head>\n <meta http-equiv=""Content-...",clueweb12-0000tw-00-00162


In [ ]:
selected_doc = warc_df.iloc[18]

In [ ]:
stripped_webpage = bsp.scrape_webpage(selected_doc['HTML_DOC'])

In [ ]:
stripped_webpage

array(['\nMy account |\r\n            Log out\n',
       'Hi are you new? Start here.',
       'SAVE UP TO 50% On selected ski, snowboard and end-of-season clothing & gear.',
       '\r\n                by\r\n                \ncheshire255\n\r\n                added 8 hours ago\n',
       '@mduscavage Yes, that link works, but you have to copy and paste it, not click the link. Both links error when you just click them. But thanks for the help!',
       '\n\nthe18thtee84\n\n\n\n5 hours ago\n\n',
       'Try this link: http://www.rei.com/Sale+and+Clearance When I used the other I kept getting an error. Great sale, thanks OP!',
       '\n\nmduscavage\n\n\n\n5 hours ago\n\n',
       'hmm some of the things on the list arent listed on sale or clearance. At first glance, fivefingers and toe socks (which I was looking for!) were not discounted at all.',
       '\n\ndodey23\n\n\n\n3 hours ago\n\n',
       'Try this link: http://www.rei.com/Sale+and+Clearance When I used the other I kept getting

### 3. NLP Preprocessing

In [2]:
spacy_processor = spacy.load("en_core_web_md")
# nlp.add_pipe("entityLinker", last=True)  # entity linker
# claucy.add_to_pipe(nlp)  # Open IE

In [3]:
spacy_doc = nlp_prep.get_nlp_doc(stripped_webpage, spacy_processor)

NameError: name 'stripped_webpage' is not defined

In [ ]:
processed_page = nlp_prep.nlp_preprocessing(spacy_doc)

### 4. Information extraction

##### 4.1 Named Entity Recognition

In [241]:
ner_page = ner.detect_entities(spacy_doc)

In [242]:
ner_page

,label,ner_type
0,SAVE UP,ORG
1,50%,PERCENT
2,clothing & gear,ORG
3,cheshire255,ORG
4,8 hours ago,TIME
5,first,ORDINAL
7,some $$,MONEY
8,REI,ORG
9,â¢Offer,PERSON
10,2/9/12 9pm,DATE


#### 4.2 Entity linking

In [243]:
warc_trec_id = [selected_doc['WARC-TREC-ID'] for x in range(len(ner_page))]

In [244]:
entities_to_link = ner_page['label'].to_list()
linked_entities = []
for entity in tqdm(entities_to_link, desc="Entity"):
    linked_entities.append(el.link_entity(entity))

Entity: 100%|██████████| 25/25 [00:14<00:00,  1.73it/s]


In [245]:
ner_page = ner_page.reset_index(drop=True)
entity_result = pd.concat([ner_page, pd.DataFrame(linked_entities), pd.DataFrame(warc_trec_id)], axis=1)

##### 4.2 Relation Extraction

In [246]:
coref_doc = cr.coref_resolution(spacy_doc)

In [247]:
relations = cre.extract_relations(coref_doc)

{}
(UP TO 50%, 8 hours ago, @mduscavage, 5 hours ago, @mduscavage, @mduscavage, OP, mduscavage, 5 hours ago, first, @mduscavage, 3 hours ago, @mduscavage, OP, mduscavage, 5 hours ago, mduscavage, 5 hours ago, first, @mduscavage, 3 hours ago, 3 hours ago, REI, 2/9/12 9, REI.com, 1-800-426-4840, REI, U.S., 2 hours ago, eckerput, Toblerone Chocolate, an hour ago, 34, 10, 4, Woot Services LLC, Deals.Deals, Deals, Deals, Deals, Deals, Woot, Inc., 2012, Dealsin, Woot, Deals, Deals, REI, Deals, REI, REI, REI, REI, Deals.Deals, Deals, Deals, Deals, five minutes)


In [248]:
# Turn list of tuples into dataframe
relations_df = pd.DataFrame(relations, columns=['relation', 'subject', 'object'])
relations_df = relations_df.dropna()

In [249]:
relations_df['object'] = relations_df['object'].apply(lambda x: x.text)
relations_df['subject'] = relations_df['subject'].apply(lambda x: x.text)

In [203]:
object_entities = relations_df['object'].to_list()
subject_entities = relations_df['subject'].to_list()
obj_ents, subj_ents = [], []
print(f'Linking object entities')
for entity in tqdm(object_entities):
    obj_ents.append(el.link_entity(entity))
print(f'Linking subject entities')
for entity in tqdm(subject_entities):
    subj_ents.append(el.link_entity(entity))

Linking object entities


100%|██████████| 105/105 [01:04<00:00,  1.64it/s]


Linking subject entities


100%|██████████| 105/105 [00:52<00:00,  2.00it/s]


Making sure all entities in the relations appear in the found entities

In [250]:
relations_df['subject_wiki'] = subj_ents
relations_df['object_wiki'] = obj_ents
relations_df.head(10)

,relation,subject,object,subject_wiki,object_wiki
2,selected,UP TO 50%,8 hours ago,https://en.wikipedia.org/wiki/Gmail,https://en.wikipedia.org/wiki/Windows_8
3,added,UP TO 50%,8 hours ago,https://en.wikipedia.org/wiki/Gmail,https://en.wikipedia.org/wiki/Windows_8
4,works Both,@mduscavage,5 hours ago,None,https://en.wikipedia.org/wiki/Grand_Theft_Auto_V
5,have to,@mduscavage,5 hours ago,None,https://en.wikipedia.org/wiki/Grand_Theft_Auto_V
6,copy,@mduscavage,5 hours ago,None,https://en.wikipedia.org/wiki/Grand_Theft_Auto_V
7,paste that,@mduscavage,5 hours ago,None,https://en.wikipedia.org/wiki/Grand_Theft_Auto_V
8,click that,@mduscavage,5 hours ago,None,https://en.wikipedia.org/wiki/Grand_Theft_Auto_V
9,works Both,@mduscavage,5 hours ago,None,https://en.wikipedia.org/wiki/Grand_Theft_Auto_V
10,click that,@mduscavage,5 hours ago,None,https://en.wikipedia.org/wiki/Grand_Theft_Auto_V
11,Try this,5 hours ago,@mduscavage,https://en.wikipedia.org/wiki/Grand_Theft_Auto_V,None


In [251]:
relations_df = relations_df.loc[(relations_df['object_wiki'].isin(linked_entities)) | (relations_df['subject_wiki'].isin(linked_entities)) , :]

In [252]:
warc_trec_id_rel = [selected_doc['WARC-TREC-ID'] for x in range(len(relations_df))]
relations_df = relations_df.reset_index(drop=True)
relations_result = pd.concat([relations_df, pd.DataFrame({"warc_trec_id": warc_trec_id_rel})], axis=1)

In [253]:
def generate_results(entity_result, relations_result):
    # Printing entities
    entity_result.columns = ['entity_name', 'ner_type', 'wiki_link', 'warc_trec_id']
    entity_result = entity_result[['warc_trec_id', 'entity_name', 'wiki_link']]

    for entity in entity_result.iterrows():
        print(f'ENTITY:{entity[1]["warc_trec_id"]}\t{entity[1]["entity_name"]}\t{entity[1]["wiki_link"]}')
    # Printing relations
    relations_result = relations_result[['warc_trec_id', 'subject_wiki', 'object_wiki', 'relation']]
    for relation in relations_result.iterrows():
        print(f'RELATION:{relation[1]["warc_trec_id"]}\t{relation[1]["subject_wiki"]}\t{relation[1]["object_wiki"]}\t{relation[1]["relation"]}')

In [254]:
generate_results(entity_result, relations_result)

ENTITY:clueweb12-0000tw-00-00835	SAVE UP	https://en.wikipedia.org/wiki/My_Dress-Up_Darling
ENTITY:clueweb12-0000tw-00-00835	50%	https://en.wikipedia.org/wiki/50_Cent
ENTITY:clueweb12-0000tw-00-00835	clothing & gear	https://en.wikipedia.org/wiki/Clothing
ENTITY:clueweb12-0000tw-00-00835	cheshire255	https://en.wikipedia.org/wiki/2015_in_British_television
ENTITY:clueweb12-0000tw-00-00835	8 hours ago	https://en.wikipedia.org/wiki/Windows_8
ENTITY:clueweb12-0000tw-00-00835	first	https://en.wikipedia.org/wiki/George_Washington
ENTITY:clueweb12-0000tw-00-00835	some $$	https://en.wikipedia.org/wiki/Tom_Hanks
ENTITY:clueweb12-0000tw-00-00835	REI	https://en.wikipedia.org/wiki/Rei_Ayanami
ENTITY:clueweb12-0000tw-00-00835	â¢Offer	https://en.wikipedia.org/wiki/UC_Browser
ENTITY:clueweb12-0000tw-00-00835	2/9/12 9pm	https://en.wikipedia.org/wiki/9
ENTITY:clueweb12-0000tw-00-00835	2/20/12 11:59pm	https://en.wikipedia.org/wiki/List_of_Miraculous:_Tales_of_Ladybug_&_Cat_Noir_episodes
ENTITY:clueweb12-

### 5. Linking